<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Camera-Calibration" data-toc-modified-id="Camera-Calibration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Camera Calibration</a></span></li><li><span><a href="#Pipeline" data-toc-modified-id="Pipeline-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pipeline</a></span><ul class="toc-item"><li><span><a href="#Helper-Functions" data-toc-modified-id="Helper-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Helper Functions</a></span></li><li><span><a href="#Testing-Ground-for-Pipeline" data-toc-modified-id="Testing-Ground-for-Pipeline-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Testing Ground for Pipeline</a></span><ul class="toc-item"><li><span><a href="#Undistort" data-toc-modified-id="Undistort-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Undistort</a></span></li><li><span><a href="#Binary-Image-Thresholded" data-toc-modified-id="Binary-Image-Thresholded-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Binary Image Thresholded</a></span></li><li><span><a href="#Perspective-Transform" data-toc-modified-id="Perspective-Transform-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Perspective Transform</a></span><ul class="toc-item"><li><span><a href="#A-Test" data-toc-modified-id="A-Test-2.2.3.1"><span class="toc-item-num">2.2.3.1&nbsp;&nbsp;</span>A Test</a></span></li></ul></li><li><span><a href="#Identify-Lane-Line-Pixels" data-toc-modified-id="Identify-Lane-Line-Pixels-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Identify Lane-Line Pixels</a></span></li><li><span><a href="#Radius-of-Curvature-of-Lane-&amp;-Position-of-Vehicle-w.r.t.-Center" data-toc-modified-id="Radius-of-Curvature-of-Lane-&amp;-Position-of-Vehicle-w.r.t.-Center-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>Radius of Curvature of Lane &amp; Position of Vehicle w.r.t. Center</a></span></li></ul></li><li><span><a href="#Main-Pipeline" data-toc-modified-id="Main-Pipeline-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Main Pipeline</a></span></li></ul></li></ul></div>

In [ ]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import math
%matplotlib inline

# Camera Calibration

* Find camera matrix & distortion coefficients
* Demonstrate undistorting calibration image


In [ ]:
def calibration_undistort(images):
    '''
    Use images to calculate the camera calibration & image distortion matrices
    '''
    # Object points follow a grid, like (0,0,0), (1,0,0), (2,0,0)...
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpts = [] # 3d points in real world space
    imgpts = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for fname in images:
        # Read in each image
        img = cv2.imread(fname)

        # Find the chessboard corners
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # Add object points & image points if found
        if ret == True:
            objpts.append(objp)
            imgpts.append(corners)
     
    # Create matrix using all the images (with corners found)
    # ret, mtx, dist, rvecs, tvecs
    _, mtx, dist, _, _ = cv2.calibrateCamera(objpts, imgpts, img.shape[1:], None, None)
    return (mtx, dist)

In [ ]:
# Load all the images & use them to the matrices 
images = glob.glob('camera_cal/calibration*.jpg')
mtx, dist = calibration_undistort(images)

In [ ]:
# TEST: Plot out each image undistorted 
images = glob.glob('camera_cal/calibration*.jpg')
for fname in images:
    # Undistort the image
    img = cv2.imread(fname)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Plotting side-by-side
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(undist)
    ax2.set_title('Undistorted and Warped Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
# Save the calibration for later use
pickle.dump({'mtx': mtx, 'dist': dist}, 
            open('camera_cal/calibration_matrices.pickle', 'wb')
)

# Pipeline

For all images:

* Distortion correction image
* Create a thresholded binary image
    - Color transforms, gradients or other methods
* Perspective transform ("birds-eye view")
* Identify lane-line pixels and fit their positions with a polynomial
* Calculate radius of curvature of lane & position of vehicle with respect to center
* Result plotted down onto the road so lane area is identified clearly


## Helper Functions

In [ ]:
def read_image_from_file(img_filepath, convert=None):
    '''
    Returns the filename/filepath & image (numpy matrix) from given file path.
    '''
    img = cv2.imread(img_filepath)
    
    # Allow for color conversion
    if convert:
        img = cv2.cvtColor(img, convert)
        
    return (img_filepath, img)

def display_images(fnames_and_imgs, figsize=(24,8), cols=3, rows=None, cmap=None, convert=None):
    '''
    Display images given a list of images to display. If `rows` is `None`, 
    rows will be calculated given the number of columns `cols`
    '''
    
    # Getting the size to plot; enough rows to fit column
    if not rows:
        rows = math.ceil(len(fnames_and_imgs)/cols)
    elif rows*cols < len(fnames_and_imgs):
        print(f'Note: not all images will be displayed')
        print(f'{len(fnames_and_imgs)} images, but only {rows*cols} will be displayed')
    f, axs = plt.subplots(rows, cols, figsize=figsize)
    
    
    # Iterate over each axis to display the image
    for ax, fname_imgs_tuple in zip(axs.flat, fnames_and_imgs):
        fname, img = fname_imgs_tuple
        # Allow for color conversion
        if convert:
            img = cv2.cvtColor(img, convert)
        ax.set_title(fname)
        ax.imshow(img, cmap=cmap)
    

## Testing Ground for Pipeline

In [ ]:
# Load camera calibration matrices from earlier
cameraCalibration = pickle.load( open('camera_cal/calibration_matrices.pickle', 'rb' ) )
mtx, dist = map(cameraCalibration.get, ('mtx', 'dist'))

In [ ]:
# Read in files to convert
test_image_fnames = glob.glob('test_images/advance/*.jpg')
test_images = [read_image_from_file(img, convert=cv2.COLOR_BGR2RGB) 
                   for img in test_image_fnames]

# Display the unmodified images
display_images(test_images, cols=4, figsize=(24,8))

### Undistort

In [ ]:
# Distortion correction
def undistort_image(img, mtx, dist):
    '''
    Undistort a given image using `mtx` & `dist` matrices.
    '''
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)
    return undist_img

In [ ]:
# Test undistortion
undist_imgs = [(fn,undistort_image(i,mtx,dist)) for fn,i in test_images]
display_images(undist_imgs, cols=4)

### Binary Image Thresholded

In [ ]:
def abs_sobel_thresh(img, orient='x', ksize=3, thresholds=(0,255)):
    '''
    Using OpenCV Sobel() function, get the absolute value of the x or y 
    gradient to produce a binary image based on the kernel size and the
    given thresholds.
    '''
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Take the absolute value of x or y gradient using OpenCV Sobel() function 
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize))
    elif orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize))
    
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a binary image of 1s where threshold is met, otherwise 0s
    binary_out = np.zeros_like(scaled_sobel)
    thresh_min, thresh_max = thresholds
    binary_out[(scaled_sobel > thresh_min) & (scaled_sobel < thresh_max)] = 1

    # Return the binary image
    return binary_out

def mag_thresh(img, sobel_ksize=3, thresholds=(0,255)):
    '''
    Using OpenCV Sobel() function, get the gradient magnitude of the x and y 
    gradients to produce a binary image based on the kernel size and the
    given thresholds.
    '''
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_ksize)
    
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    
    # Create a binary image of 1s where threshold is met, otherwise 0s
    binary_out = np.zeros_like(gradmag)
    thresh_min, thresh_max = thresholds
    binary_out[(gradmag > thresh_min) & (gradmag < thresh_max)] = 1

    # Return the binary image
    return binary_out

def dir_threshold(img, sobel_ksize=3, thresholds=(0,np.pi/2)):
    '''
    Using OpenCV Sobel() function, get the gradient direction of the x and y 
    gradients to produce a binary image based on the kernel size and the
    given thresholds.    
    '''
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_ksize)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_ksize)
    
    # Calculate absolute value of the gradient direction, 
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    
    # Create a binary image of 1s where threshold is met, otherwise 0s 
    binary_out =  np.zeros_like(absgraddir)
    thresh_min, thresh_max = thresholds
    binary_out[(absgraddir > thresh_min) & (absgraddir < thresh_max)] = 1

    # Return the binary image
    return binary_out

In [ ]:
def full_binary_image(img, ksize=3, x_thres=(30,100), y_thres=(30,100), 
                      mag_thres=(30,100), dir_thres=(0,np.pi/2)):
    '''
    Using OpenCV Sobel() function, get the gradient direction of the x and y 
    gradients to produce a binary image based on the kernel size and the
    given thresholds.   
    '''
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(img, orient='x', ksize=ksize, thresholds=x_thres)
    grady = abs_sobel_thresh(img, orient='y', ksize=ksize, thresholds=y_thres)
    mag_binary = mag_thresh(img, sobel_ksize=ksize, thresholds=mag_thres)
    dir_binary = dir_threshold(img, sobel_ksize=ksize, thresholds=dir_thres)

    # Create a binary image of 1s where criteria is met, otherwise 0s 
    combined = np.zeros_like(dir_binary)
    # Gadients in x or y are met or the magnitue & direction are met
    combined[
        ((gradx == 1) & (grady == 1)) |  
        ((mag_binary == 1) & (dir_binary == 1))
    ] = 1
    
    return combined

In [ ]:
# Test out the performance of the full binary threshold
binary_imgs = [
    (
        fn,
        full_binary_image(i,
                          ksize=11,
                          x_thres=(30,100), 
                          y_thres=(30,100),
                          mag_thres=(40,100), # (30,100) might be better
                          dir_thres=(np.pi/4, np.pi*2/5)
#                           dir_thres=(np.pi/6, np.pi*5/6)
        )
    ) 
        for fn,i in test_images
]

display_images(binary_imgs, cols=4, cmap='gray')

In [ ]:
#TODO: HLS color threshold

### Perspective Transform

In [ ]:
def create_lane_guides(img, lines, offset=200, line_color=(0,255,0), show=True):
    '''
    Return a perspective transform image based on the provided quadrilateral
    representing lane lines in the original image.
    
    Note lines must go like [Bottom-Left, Top-Left, Top-Right, Bottom-Right].
    '''    
    copy = img.copy()

    line_thickness = 2
    # Create a line on image for each pair of points(that defines a line)
    for pt0,pt1 in zip(lines, lines[1:]):
        cv2.line(copy, pt0, pt1, line_color, line_thickness)

    # Display the lane guides with original image
    if show:
        fig, ax = plt.subplots(figsize=(40, 20))
        ax.imshow(copy)
    
    # Getting image and the relevant measurements
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_size = (gray.shape[1], gray.shape[0])
    
    # Getting the image destination
    # Bottom-Left, Top-Left, Top-Right, Bottom-Right
    dst = np.float32([
        [offset, img_size[1]],
        [offset, 0],
        [img_size[0]-offset, 0],
        [img_size[0]-offset, img_size[1]]
    ])
    
    # Source from the lane guide lines
    src = np.zeros(shape=(len(lines),2), dtype='float32')
    for i,line_pts in enumerate(lines):
        src[i,:] = line_pts
    
 
    # Unwarp the image
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(undist, M, img_size)

    if show:
        fig, ax = plt.subplots(figsize=(40, 20))
        ax.imshow(warped)
    
    return warped

#### A Test

In [ ]:
# Geetting a straight lines test (straight_lines1.jpg)
undist = undist_imgs[-1][1]

In [ ]:
# Create guides for the typical lanes needed
y_bottom = 720
y_top = 455

# Bottom-Left, Top-Left, Top-Right, Bottom-Right
lines = [
    (190, y_bottom), # bottom left
    (585, y_top), # top left
    (705, y_top), # top right
    (1130, y_bottom) # bottom right
]

In [ ]:
warped = create_lane_guides(undist, lines, show=True);

### Identify Lane-Line Pixels

### Radius of Curvature of Lane & Position of Vehicle w.r.t. Center

## Main Pipeline